In [140]:
import pandas as pd
import nose.tools
from pandas.api.types import is_numeric_dtype
# Write your imports here

# Data Tidying and Cleaning Lab
## Reading, tidying and cleaning data. Preparing data for exploration, mining, analysis and learning

### Problem 1. Read the dataset (2 points)
The dataset [here](http://archive.ics.uci.edu/ml/datasets/Auto+MPG) contains information about fuel consumption in different cars.

Click the "Data Folder" link and read `auto_mpg.data` into Python. You can download it, if you wish, or you can read it directly from the link.

Give meaningful (and "Pythonic") column names, as per the `auto_mpg.names` file:
1. mpg
2. cylinders
3. displacement
4. horsepower
5. weight
6. acceleration
7. model_year
8. origin
9. car_name

In [141]:
# pd.set_option('display.max_rows', None)
mpg_data = pd.read_fwf("data/auto-mpg.data")
mpg_data.columns = [
    'mpg',
    'cylinders',
    'displacement',
    'horsepower',
    'weight',
    'acceleration',
    'model_year',
    'origin',
    'car_name',
]

In [142]:
nose.tools.assert_is_not_none(mpg_data)

Print the first 4 rows in the dataset to get a feel of what it looks like:

In [143]:
mpg_data.head(4)

mpg  cylinders  displacement horsepower  weight  acceleration  model_year  \
0  15.0          8         350.0      165.0  3693.0          11.5          70   
1  18.0          8         318.0      150.0  3436.0          11.0          70   
2  16.0          8         304.0      150.0  3433.0          12.0          70   
3  17.0          8         302.0      140.0  3449.0          10.5          70   

   origin              car_name  
0       1   "buick skylark 320"  
1       1  "plymouth satellite"  
2       1       "amc rebel sst"  
3       1         "ford torino"

### Problem 2. Inspect the dataset (1 point)
Write a function which accepts a dataset and returns the number of observations and features in it, like so: 

``` 10 observations on 15 features```

Where 10 and 15 should be replaced with the real numbers. Test your function with the `auto_mpg` dataset.

Make sure the function works with other datasets (don't worry about "1 features" or "1 observations", just leave it as it is).

In [144]:
def observations_and_features(dataset):
    """
    Returns the number of observations and features in the provided dataset
    """
    observations = len(dataset.index)
    features = len(dataset.columns)
    return "{} observations on {} features".format(observations, features)

In [145]:
print(observations_and_features(mpg_data))

397 observations on 9 features


In [146]:
billboard_data = pd.read_csv("data/billboard.csv")
print(observations_and_features(billboard_data))

317 observations on 81 features


Inspect the data types for each column.

In [147]:
mpg_data.dtypes

mpg             float64
cylinders         int64
displacement    float64
horsepower       object
weight          float64
acceleration    float64
model_year        int64
origin            int64
car_name         object
dtype: object

### Problem 3. Correct errors (1 point)
The `horsepower` column looks strange. It's a string but it must be a floating-point number. Find out why this is so and convert it to floating-point number.

In [148]:
is_numeric_dtype(mpg_data.horsepower)

False

In [149]:
mpg_data.horsepower = mpg_data.horsepower.replace('?',np.nan).astype(float)
is_numeric_dtype(mpg_data.horsepower)

True

In [150]:
nose.tools.assert_equal(mpg_data.horsepower.dtype, "float64")

### Problem 4. Missing values: inspection (1 point)
We saw that the `horsepower` column contained null values. Display the rows which contain those values. Assign the resulting dataframe to the `unknown_hp` variable.

In [156]:
def get_unknown_hp(dataframe):
    """
    Returns the rows in the provided dataframe where the "horsepower" column is NaN
    """
    unknown_hp = None
    if 'horsepower' in dataframe.columns:
        unknown_hp = dataframe[dataframe['horsepower'].isna()]
    return unknown_hp

In [157]:
cars_with_unknown_hp = get_unknown_hp(mpg_data)
print(cars_with_unknown_hp)

      mpg  cylinders  displacement  horsepower  weight  acceleration  \
31   25.0          4          98.0         NaN  2046.0          19.0   
125  21.0          6         200.0         NaN  2875.0          17.0   
329  40.9          4          85.0         NaN  1835.0          17.3   
335  23.6          4         140.0         NaN  2905.0          14.3   
353  34.5          4         100.0         NaN  2320.0          15.8   
373  23.0          4         151.0         NaN  3035.0          20.5   

     model_year  origin                car_name  
31           71       1            "ford pinto"  
125          74       1         "ford maverick"  
329          80       2  "renault lecar deluxe"  
335          80       1    "ford mustang cobra"  
353          81       2           "renault 18i"  
373          82       1        "amc concord dl"  


### Problem 5. Missing data: correction (1 point)
It seems like the `NaN` values are a small fraction of all values. We can try one of several things:
* Remove them
* Replace them (e.g. with the mean power of all cars)
* Look up the models on the internet and try our best guess on the power

The third one is probably the best but the first one will suffice since these records are too few. Remove those values. Save the dataset in the same `mpg_data` variable. Ensure there are no more `NaN`s.

In [170]:
mpg_data = mpg_data.dropna()
len(get_unknown_hp(mpg_data))

0

In [ ]:
nose.tools.assert_equal(len(get_unknown_hp(mpg_data)), 0)

### Problem 6. Years of production (1 + 1 points)
Display all unique model years. Assign them to the variable `model_years`.

In [171]:
def get_unique_model_years(dataframe):
    """
    Returns the unique values of the "model_year" column
    of the dataframe
    """
    model_years = None
    if "model_year" in dataframe.columns:
        model_years = dataframe.model_year.unique()
    return model_years

In [172]:
model_years = get_unique_model_years(mpg_data)
print(model_years)

[70 71 72 73 74 75 76 77 78 79 80 81 82]


These don't look so good. Convert them to real years, like `70 -> 1970, 71 -> 1971`. Replace the column values in the dataframe.

In [174]:
mpg_data.model_year = mpg_data.model_year + 1900

In [175]:
model_years = get_unique_model_years(mpg_data)
print(model_years)

[1970 1971 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1982]


### Problem 7. Exploration: low-power cars (1 point)
The data looks quite good now. Let's try some exploration.

Write a function to find the cars which have the smallest number of cylinders and print their model names. Return a list of car names.

In [212]:
def get_model_names_smallest_cylinders(dataframe):
    """
    Returns the names of the cars with the smallest number of cylinders
    """
    car_names = None
    if "cylinders" in dataframe.columns:
        car_names = dataframe[dataframe.cylinders == dataframe.cylinders.min()].car_name
    return car_names

In [213]:
car_names = get_model_names_smallest_cylinders(mpg_data)
print(car_names)
nose.tools.assert_true(car_names.shape == (4,) or car_names.shape == (4, 1))

70     "mazda rx2 coupe"
110          "maxda rx3"
242         "mazda rx-4"
333      "mazda rx-7 gs"
Name: car_name, dtype: object


### Problem 8. Exploration: correlations (1 point)
Finally, let's see some connections between variables. These are also called **correlations**.

Find how to calculate correlations between different columns using `pandas`.

**Hint:** The correlation function in `pandas` returns a `DataFrame` by default. You need only one value from it.

Create a function which accepts a dataframe and two columns and prints the correlation coefficient between those two columns.

In [214]:
def calculate_correlation(dataframe, first_column, second_column):
    """
    Calculates and returns the correlation coefficient between the two columns in the dataframe.
    """
    correlation = None
    if first_column in dataframe.columns and second_column in dataframe.columns:
        correlation = dataframe[first_column].corr(dataframe[second_column])
    return correlation

In [216]:
hp_weight = calculate_correlation(mpg_data, "horsepower", "weight")
print("Horsepower:Weight correlation coefficient:", hp_weight)
nose.tools.assert_almost_equal(hp_weight, 0.864537737574, delta = 0.01)


Horsepower:Weight correlation coefficient: 0.8643969121835688
